<br><FONT size ="10"> Basics of mobile robotics project report</FONT>
<p>Quentin Delfosse
    <br>Bastien Darbellay
    <br>Daniel Tataru
    <br>Hippolyte Rauch</p>
    
<h1>Introduction</h1>
<p>The goal of this project is to program a small mobile robot (a Thymio) and make it navigate from its starting point to his goal position by avoiding any obstacle.</p>
<p>In our case, our robot is cross_country skiing on a flat field and has to join his friends to have a mulled wine. However on his way there are forests through which the robot cannot pass. Finally, a Yeti can come out of the woods at any time and the robot must avoid it.</p>
    <p><br>The project is divided in four components:
    <ul>
        <li>Vision</li>
        <li>Global navigation</li>
        <li>Filtering</li>
        <li>Local navigation</li>
        <li>Motion control</li>
    </ul>
    </p>

<hr>
<h1>Vision</h1>
<h2>Presentation</h2>
<p>The purpose of the vison component is to analyse the environment surrounding the robot with a camera placed above the map. For each type of object that we want to detect we definite a different color:
    <ul>
        <li>The obstacles (<FONT color="green">green</FONT>)</li>
        <li>The robot's position and angle (<FONT color="blue">blue</FONT>)</li>
        <li>The goal position (<FONT color="purple">purple</FONT>)</li>
    </ul>
</p>
<p>First we apply a mask for a certain color range, then we detect the shapes and finally we keep only the vertex.</p>
<p>We use <i>OpenCV</i> library which is very usefull for image analysis and <i>matplotlib</i> to display the images</p>

<h2>Color detection</h2>
<p> Because of the variations of luminosity seen by the camera we need for each color to define a range of values. OpenCV's default color space is the BGR color space and it is not really intuitive to define ranges. Therefore we convert the images to a much more intuitive color space: HSV. This can be done in openCV with <FONT style="font-family:courier;">img_HSV = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)</FONT>. In HSV the pixels are encoded with three values:
    <ul>
        <li>Hue (0-179)</li>
        <li>Saturation (0-255)</li>
        <li>Value (0-255)</li>
    </ul>
</p>
<img src="HSB.png" width="200" />
     
<p> The colors noted above are choosen such as their are sufficiently far one from the other in the HSV color space and hence we do not have conflict between them.</p>

<p> The next step now is to defin this ranges. We did a lot of tests and ended with :

In [3]:
GREEN_MIN = np.array([0, 50, 0])
GREEN_MAX = np.array([58, 255, 255])
PURPLE_MIN = np.array([150, 100, 80])
PURPLE_MAX = np.array([170, 255, 255])
BLUE_MIN = np.array([90, 60, 40])
BLUE_MAX = np.array([135, 255, 255])

We can know apply the mask on our image with the function <FONT style="font-family:courier;">mask_green=cv2.inRange(img, GREEN_MIN, GREEN_MAX)</FONT>

<h3> Shapes detection </h3>
<p>This part deals with creating the map and finding Thymio’s position as well as the end goal.
    
Thanks to the masks provided by the color_detection algorithm, we rely on the <FONT style="font-family:courier;">image_to_vertices()</FONT> function to return polygons for the different patterns looked for. This function is therefore called 3 times and provides us with the edges of the Thymio, the end goal and the obstacles. A <FONT style="font-family:courier;">smoothen_contours()</FONT> function is called to set a threshold for edge detection.</p>

In [6]:
def image_2_vertices(image,epsilon_cst):

contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
smooth_contours = smoothen_contours(contours,epsilon_cst)
cv2.drawContours(image, smooth_contours, -1, (200, 200, 200), 3)
polys = get_vertices(image, smooth_contours)

return polys

IndentationError: expected an indented block (<ipython-input-6-98008b8d9ca8>, line 3)

Given the contours of the Thymio and the final goal, we now need to compute their position. 2 functions serve this purpose:

* <FONT style="font-family:courier;">find_thymio()</FONT>

* <FONT style="font-family:courier;">find_destination()</FONT>

Apart from being color differentiated, our robot has a triangle on its top while the goal is a square. The triangular shape helped to determine the orientation of the Thymio.

Details for the computation of the center of the triangle hereunder:

<FONT style="font-family:courier;">find_thymio()</FONT>:<img src="Thymio_triangle.jpeg" width="700" />

Nevertheless, a criticism could be made on the high dependence on the lighting conditions of the algorithm used. This forced us to work in a controlled environment with blinds closed and artificial lights to always work under the same conditions.

<hr>
<h1>Global navigation</h1>
    <p>In order to compute the optimal path between our now-know Thymio position and its target (the mulled wine), we used a global planner to takes into consideration pre-planned obstacles.

Our path is computed in the global_planner file. The main function responsible for returning the path is <FONT style="font-family:courier;">global_path()</FONT>.</p>

<h2>Obstacles increase</h2>

First, some changes need to be made to our obstacles. Indeed, the latter need to be increased so that our cross-country skier can fit between them.

The <FONT style="font-family:courier;">increase_coordinates()</FONT> function is in charge of this task by means of the <i>pyclipper</i> library.

In [ ]:
def increase_coordinates(polygon):

subj = []

for p in polygon:
    subj.append((math.floor(p.x),math.floor(p.y)))
    subj = tuple(subj)
    pco = pyclipper.PyclipperOffset()
    pco.AddPath(subj, pyclipper.JT_MITER, pyclipper.ET_CLOSEDPOLYGON)
    solution = pco.Execute(OBST_INCREASE)
    polygon = []

for i in range(len(solution[0])):
    polygon.append(Point(solution[0][i][0],solution[0][i][1]))

return polygon

This library uses the border normal vectors of each polygon to increase them by a factor <FONT style="font-family:courier;">OBST_INCREASE</FONT> defined in our <FONT style="font-family:courier;">constants.py</FONT> file. It is set as the width of the Thymio to avoid collision between the two.

<h2>Obstacle merge</h2>

Then, in case 2 obstacles are closer than the width of the robot or overlap (after borders increase), we merge them thanks to the <FONT style="font-family:courier;">merge_obstacles</FONT> file that relies on another library: <i>shapely</i>. This file uses the crossing lines of 2 polygons to recompute the borders of a single polygon.

In [ ]:
def unify(p1,p2):
    
if p1 == []:
    return p2
elif p2 == []:
    return p1
else:
    polygon1 = vg2shapely(p1)
    polygon2 = vg2shapely(p2)
    polygons = [polygon1, polygon2]
    print(polygons)
    u = unary_union(polygons)
return shapely2vg(u)

<h2>Global path computation</h2>

Only now can we compute the path. For this purpose, we used a visibility graph based on the lines connecting all our obstacle vertices to determine the smallest distance from point A to final point B. The <i>pyvisgraph</i> package was used for this task.

This function is also used to draw the map obstacles and the visibility graph on the provided image by calling the <FONT style="font-family:courier;">draw_polygon()</FONT> and <FONT style="font-family:courier;">draw_visible_vertices()</FONT> functions.

Note: The global path is only computed once at the very beginning or in case the final position has changed from a certain value.

<hr>
<h1>Filtering</h1>
    <p>In order to estimate the Thymio's position and velocity at a given time using various available measurements, we merge all this data using a Bayesian filter, in particular a Kalman filter, to obtain the best possible estimation with the lowest covariance possible.</p>
    
<h2> State-space model</h2>
<p>The first step is therefore the definition of our state-space model (in the discrete domain), which can be written as follows:


$$ x_{k+1} = A*x_k + q $$
$$ y_k = C*x_k + r $$

Where $ x_k = \begin{pmatrix}x\\ v_x\\ y\\ v_y\\ \theta\\ \omega  \end{pmatrix}$, 
$ A = \begin{pmatrix}1&Ts&0&0&0&0\\ 0&1&0&0&0&0\\ 0&0&1&Ts&0&0\\ 0&0&0&1&0&0\\ 0&0&0&0&1&Ts\\ 0&0&0&0&0&1  \end{pmatrix}$,
$ y = \begin{pmatrix}v_x\\ v_y\\ \omega  \end{pmatrix}$ if video camera is obstructed and
$ y = \begin{pmatrix}x\\y\\ \theta\\v_x\\ v_y\\ \omega  \end{pmatrix}$ if video camera can detect the Thymio

Therefore depending on the case C is a 3x6 or 6x6 matrix extracting given values:

$ C = \begin{pmatrix}0&1&0&0&0&0\\ 0&0&0&1&0&0\\ 0&0&0&0&0&1 \end{pmatrix}$ if camera position measurement is not available and

$ C = \begin{pmatrix}1&0&0&0&0&0\\ 0&0&1&0&0&0\\ 0&0&0&0&1&0\\0&1&0&0&0&0\\ 0&0&0&1&0&0\\ 0&0&0&0&0&1 \end{pmatrix}$ otherwise.

q is the disturbance with a covariance matrix Q and r is the measurement noise with covariance matrix r.

The available measurements regarding the Thymio's speed are the wheel speeds, however we have to convert them to absolute x and y speeds as well as $ \omega $ in order to not have non-linearities in our Kalman filter state-space model, which would need to be dealt using an Extended Kalman filter. Our approach seemed precise enough, considering that the camera gives much better measurements and that it is assumed that the camera would not be obstructed for more than a few seconds, and so our estimations remain within a valid range.
The conversion factors were computed via testing the Thymio for given rotational and translational speeds.
We use the rotation matrix shown during the course to perform the operation at the beginning of our update function:</p>

In [4]:
speed_conv_factor = 0.20#From Thymio linear speed to mm/s
rot_conv_factor = -0.0031 #From Thymio rotational speed to rad/s

motor_left_speed = variables["motor_left_speed"]
motor_right_speed = variables["motor_right_speed"]
speed_x = (speed_conv_factor*(motor_left_speed+motor_right_speed)/2.0) * np.cos(theta)
speed_y = (speed_conv_factor*(motor_left_speed+motor_right_speed)/2.0) * np.sin(theta)
speed_w = rot_conv_factor*(motor_right_speed-motor_left_speed)/(2.0)

NameError: name 'variables' is not defined

<p>Initially we thought of including the acceleration measurements from the Thymio's accelerometer to get even more data to improve our final estimation, however after some initial tests it was decided to not use these values as the low resolution didn't bring any significant improvements, which is also partially due to the fact that the accelerations are quite low in our motion control part of the software meaning we wouln't obtain any meaningful results.</p>

<h2> Covariance matrices definition</h2>

The next step in order to design the Kalman filter is the measurement of the noise and disturbance covariance matrices in order to estimate how precise is our model and how noisy are our measurements. Two main assumptions were made during this part:
1. The covariance matrices are diagonal, meaning each measurement and disturbance is independent, which is not very accurate mathematically (as our speed measurements directly depend on the angle, and the camera is used for the three position measurements (x, y and $ \theta $), and our in our model in realty $ v_x $ are coupled with $ \theta $
2. The distributions are assumed Gaussian for all values


In order to measure all the values, we perform various measurements on the Thymio. Then we split the values between the model covariance matrix Q and the measurement covariance matrix.

For the position given by the camera, we ask the Thymio to perform the same movement starting at the exact same point (or as close as possible) by moving in a straight line (assuming variance in x and y are identical) during a series of repetitions, then computing the variance of the final position given. The same is done for the measurement variance of $ \theta $ where we ask the Thymio to turn on itself for 2 turns (as more yields in cable management issues) during a number of repetitions then computing the variance of the final measurements.

For the speed we proceed in a similar fashion but we just send the Thmyio a constant linear (or rotational) speed and measure all values of during a certain period of time and compute variances (similarly to what was done during the course for a 1D speed).

After this is done we further test these values on the Thymio with and without camera vision to further improve our Kalman filter performance and we end up with the following covariance matrices for the disturbance and noise:


$$ Q = \begin{pmatrix}6.1&0&0&0&0&0\\ 0&6.1&0&0&0&0\\ 0&0&6.1&0&0&0\\0&0&0&6.1&0&0 \\0&0&0&0&0&6.1\\0&0&0&0&10.2&0  \end{pmatrix} $$

$$ R = \begin{pmatrix}0.5&0&0&0&0&0\\ 0&6.1&0&0&0&0\\ 0&0&0.5&0&0&0\\ 0&0&0&6.1&0&0 \\0&0&0&0&0.5&0\\0&0&0&0&0&4 \end{pmatrix} $$ 
or 
$$ R = \begin{pmatrix}6.1&0&0\\ 0&6.1&0\\ 0&0&4 \end{pmatrix} $$ 

Depending whether we have position measurements from the camera or not respectively.

<h2> Full Kalman filter function</h2>

The last step of the Kalman filter definition is the synchronization of the variables in order to obtain meaningful results from our state-space model. This was a bit tricky as the time needed to execute a full loop of vision, planning, estimation and motion control (all the while using the local planner and obstacle avoidance) wasn't always fully constant. The sampling time was therefore measured and fine-tuned to obtain precise results, and the sampling time was set to $T_s = 0.3 s$

We have all the necessary values to implement the full kalman filter function as shown in the following cell:

In [ ]:
def kalman_filter(Ts, speed_x, speed_y, speed_w ,x_est_prev, P_est_prev, vision=0, pos_x_m=0.0, pos_y_m=0.0, theta_m=0):

    N = 6
    A = np.array([[1, Ts, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, Ts, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, Ts], [0, 0, 0, 0, 0, 1]])
    #State-space model (with acceleration, C to be modified as well)
    #A = np.array([[1, Ts, Ts*Ts/2.0, 0, 0, 0, 0, 0], [0, 1, Ts, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, Ts, Ts*Ts/2.0, 0, 0], [0, 0, 0, 0, 1, Ts, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, Ts], [0, 0, 0, 0, 0, 0, 0, 1]])
    #N = 8

    Q = 6.1*np.identity(N)
    Q[4][4] = 10.2


    if vision==1:
        y = np.array([[pos_x_m], [pos_y_m], [theta_m], [speed_x], [speed_y], [speed_w]], dtype=float)
        M = 6
        C = np.array([[1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1]])
        R = 6.1*np.identity(M) #To update in case of measurement errors
        R[0][0] = 0.5
        R[2][2] = 0.5
        R[4][4] = 0.5
        R[5][5] = 4
    else:
        y = np.array([[speed_x], [speed_y], [speed_w]], dtype=float)
        M = 3
        C = np.array([[0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1]])
        print('TRYING TO ESTIMATE THYMIO POSITION')
        R = 6.1*np.identity(M) #To update in case of measurement errors
        R[2][2] = 4

    

    x_est_a_priori = np.dot(A, x_est_prev); #A priori estimate
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T)) + Q

    i = y - np.dot(C, x_est_a_priori) #Measurement residual
    S = np.dot(C, np.dot(P_est_a_priori, C.T)) + R
    K = np.dot(P_est_a_priori, np.dot(C.T, np.linalg.inv(S))) #Kalman gain
    x_est = x_est_a_priori + np.dot(K,i) #A posteriori estimate
    P_est = P_est_a_priori - np.dot(K,np.dot(C, P_est_a_priori))

    return x_est, P_est

<p>It is important to note that the initial estimation values are set to 0 in the constants.py folder as well as the initial estimation covariance matrix $P_{est0}$ is set to 10 times the identity matrix in an arbitrary fashion.

Finally, we use the update position function at each iteration in our Main.py algorithm to estimate the position based on the given values:</p>

In [7]:
def update_position(Ts, pos_x_m, pos_y_m, theta_m, vision, motor_left_speed, motor_right_speed):

    global x_est_prev, P_est_prev, end, start

    speed_x = (speed_conv_factor*(motor_left_speed+motor_right_speed)/2.0) * np.cos(x_est_prev[4])
    speed_y = (speed_conv_factor*(motor_left_speed+motor_right_speed)/2.0) * np.sin(x_est_prev[4])
    speed_w = rot_conv_factor*(motor_right_speed-motor_left_speed)/(2.0)

    x_est, P_est = kalman_filter(Ts, speed_x, speed_y, speed_w ,x_est_prev, P_est_prev, vision, pos_x_m, pos_y_m, theta_m)

    x_est_prev = x_est
    P_est_prev = P_est

    print('OUTPUT', x_est)

    return x_est

<h1>Local navigation</h1>

The local path planner is based on a potential field, where the goal is attractive and the obstacles repulsive. This enables a very general approach to avoid obstacles that were not detected by the global path planner beforehand in all kinds of scenarios. The target of the local path planner corresponds to the Thymio’s next waypoint which is computed and provided by the global path planner. The goal first must be transformed into the robot’s reference frame since the latter differs from the global one (see figure below). Only then, the potential fields of the obstacles and the goal can be generated and summed up.

<img src="reference_frames.png" width="400" />

To compute the potential field created by the obstacles we use the horizontal proximity sensors on the robot’s front. Every time one of those sensors is detecting an obstacle the robot will convert the detected point into coordinates matching its reference frame by considering the position and orientation of each sensor with respect to the center of the robot.

The limitation of using a potential field approach with this robot is that every sensor only provides a measurement of a point in space, potentially resulting in a field that will guide the robot through two detected points even if those two points are part of the same obstacle and, hence blocking that path. To avoid this scenario, we assume that all detected points are part of the same obstacle and inflate them to create a continuous obstacle, removing the gap between the points. To do so, we make use of Bresenham’s algorithm which computes the points on our 2-dimensional raster that approximate the best a straight line between the two detected points.

<img src="Ex_inflated_obstacle.png" width="700" />

After computing the whole potential field, the trajectory is determined by taking the mean value of all force vectors contained in the area defined by the Thymio’s dimensions and the inflated obstacles. This allows the robot to evade the obstacles little by little while decreasing its turning radius as he’s approaching it, resulting in a smooth obstacle avoidance.

For this part of the code, we used the library <i>PyBresenham (https://pypi.org/project/PyBresenham/)</i> to compute the points inflating the obstacles.

<hr>
<h1>Motion control</h1>

This file is used to control the motors of the Thymio given a destination that was either computed by the global planner or the local one.

It relies on a simple proportional gain (K) controller with error estimation between our current angle and the destination’s. The further away we are in angular value from our desired goal, the higher the angular velocity command will be to align the Thymio and its goal.

This function is also used to update the path followed by removing the next point to reach if our distance from it is smaller than a threshold value.

This will allow the main file to stop when no points to reach are left in the global path list.

While the simplicity of this code has proven very helpful in debugging the overall motion control of the Thymio, a limitation of this control lies in the oscillatory behavior of the robot for targets being placed at its rear. Angular ranges could have solved this problem, but time limitations forced us to focus on improving other critical aspects of the project first.
    

In [ ]:
def motion_control(node, pos, phi, next_step, path, goal_threshold, motor_left_speed, motor_right_speed):

if math.sqrt((path[0].y-pos.y)**2+(path[0].x-pos.x)**2) > goal_threshold: # Not at goal yet, keep moving in the desired position
    phi_d = math.atan2(next_step[1]-pos.y,next_step[0]-pos.x)
    w = K*(phi_d-phi)
    motor_left_speed = 100*(v0/R + w*L/(2*R))
    motor_right_speed = 100*(v0/R - w*L/(2*R))
    node.send_set_variables(motors(int(motor_left_speed), int(motor_right_speed)))
elif goal_threshold == FINAL_GOAL: # Arrived at final goal -> stops motors
    motor_left_speed = 0
    motor_right_speed = 0
    node.send_set_variables(motors(motor_left_speed, motor_right_speed))
    path.pop(0)
elif goal_threshold == STEP_GOAL: # Updates goal to next one
    path.pop(0)
    
return [motor_left_speed, motor_right_speed]

<hr>
<h1>Merging all the codes together</h1>

Finally, all these files need to work together. To do so, we implemented a main algorithm directly in the ClientAsync() function. This allowed us to update our values of proximity sensors on an infinite loop that only stops when reaching the final target.

The sequence of action is the following:

* Getting the current camera feed image

* Retrieving the color masks

* Computing the polygons (thymio, obstacles and end goal)

* Finding the position of the Thymio and its target

* Computing the global path (only for the first iteration or if the goal has changed position)

* Calling the local planner and updating the global path with a deviation if the proximity sensors detect an unplanned obstacle

* Controlling the motors to reach the next point in the planner

* Displaying the map, thymio position, end goal and path on the camera feed.

<hr>
<h1>Conclusion</h1>

Compared to the e-puck project (MT-BA6), we felt that this robot allowed for a higher understanding of what control represents, especially regarding the autonomy of a robot. The foundations taught during the semester were easier to grasp once applied to a real robot and therefore the team is very glad to have had the opportunity to discover the basics of mobile robotics in a practical project. Such projects always tend to test our resolve and we think that in

addition to the theoretical benefit of this project, this taught us a lot as a group on how to separate the workload and solve issues under pressure of a deadline.

In the end, our skier finally made its way to his mulled wine rendez-vous and we believe that we were as happy as him to see him arrive nicely at the desired position, having avoided a yeti and lost his vision on the way.

<h1> Main program</h1>

Please run the main.py file that you will find in the zip folder